<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
!pip install clickhouse_driver --quiet
from clickhouse_driver import Client
from google.colab import userdata
!pip install itables --quiet
from itables import show
import ipywidgets as widgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.6 MB/s eta 0:00:00


In [6]:
# connection requisits
host = 'clickhouse.lab.karpov.courses'
dbase = 'default'
port = 9000
user = 'student'
password = userdata.get('ch2020_pwd')

In [8]:
client = Client(host=host, port=port, user=user,
                password=password, database=dbase)

### Задание 4.1
Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t). <br>
•	host_response_rate – частота ответа <br>
•	host_is_superhost – является ли суперхозяином

In [14]:
sql = f"""
SELECT
  host_is_superhost,
  avg(host_response_rate) as avg_host_response_rate
FROM
(SELECT
  DISTINCT host_id,
  host_is_superhost,
  toUInt8OrNull(replaceAll(host_response_rate, '%', '')) AS host_response_rate
FROM
  listings)
GROUP BY host_is_superhost
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,host_is_superhost,avg_host_response_rate
0,,NaN
1,t,98.000000
2,f,89.398451


### Задание 4.2

Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id.

In [15]:
sql = f"""
SELECT
  host_id,
  AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as avg_host_price,
  groupArray(id) as ids
FROM
  listings
GROUP BY
  host_id
HAVING length(ids) > 2
ORDER BY avg_host_price DESC, host_id DESC
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,host_id,avg_host_price,ids
0,187655517,6000.0,"[25757977, 25759146, 25802565, 25802651, 25802..."
